---

# **FableForge: Creating Picture Books with OpenAI, Replicate, and Deep Lake**
In this lesson, we are going to delve into a use case of AI technology in the creative domain of children's picture book creation in a project called "FableForge", leveraging both OpenAI GPT-3.5 LLM for writing the story and Stable Diffusion for generating images for it.

## **Introduction**
This lesson’s project, FableForge, is an application that generates picture books from a single text prompt. It utilizes the power of OpenAI's language model GPT-3.5 to write the story. Then, the text is transformed into visual prompts for Stable Diffusion, an AI that creates corresponding images, resulting in a complete picture book. The data, both text and images, are then stored in a Deep Lake dataset for easy analysis.

The article guides us through the steps of building FableForge, detailing the challenges, successes, and methodologies adopted. You will learn how the team leveraged the “function calling” feature newly introduced by OpenAI, which is used in this project specifically to structure text data suitable for Stable Diffusion, a task that initially proved difficult due to the model's tendency to include non-visual content in the prompts. We’ll see how to overcome this by using a function providing structured, actionable output for external tools.

We'll delve into each component of FableForge, including the generation of text and images, combining them into a book format, storing the data into Deep Lake, and finally presenting it all through a user-friendly interface with Streamlit. We'll explore the process of text generation, extracting visual prompts, assembling PDFs, and uploading the data to Deep Lake.

By the end of this lesson, you'll gain a comprehensive understanding of how various AI tools and methodologies can be effectively integrated to overcome challenges and open new frontiers in creative domains.



---

## **Meet FableForge, AI Picture Books Generator powered by OpenAI, LangChain, Stable Diffusion, & Deep Lake**
Imagine a world where children's picture books are created on-demand by children from a single prompt. With each generated image, the text and prompt pairs are stored for further finetuning if the child likes the story - to fit one human's imagination perfectly.

This is the grand vision of FableForge.

FableForge is an open-source app that generates children's picture books from a single prompt. First, GPT-3.5/4 is instructed to write a short children's book. Then, using the new function calling feature OpenAI just announced, the text from each book page is transformed into a prompt for Stable Diffusion. These prompts are sent to Replicate, corresponding images are generated, and all the elements are combined for a complete picture book. The matching images and prompts are stored in a Deep Lake vector database, allowing easy storing and visualizing of multimodal data (image and text pairs). Beyond that, the generated data can be streamed to machine learning frameworks in real time while training, to finetune our generative AI model. While the latter is beyond the scope of this example, we'd love to cover how it all works together.

---

## **What Did and Didn't Work while Building FableForge?**
Before we look at the exact solution we eventually decided on, let's take a glance at the approaches that didn't work and what we learned from them:

## **Didn't Work: Instructing GPT-4 To Generate Stable Diffusion Prompts**
Initially, it seemed like it might be possible to send the LLM the text of our book and tell it to generate a prompt for each page. However, this didn't work for a few reasons:

- **Stable Diffusion released in 2022:** While it might seem like Stable Diffusion is already "old news", to GPT-3.5 and GPT-4 it's in the future. Look at GPT-4's response to the question, "What is Stable Diffusion?":What is Stable Diffusion? Nobody Knew in 2021

- **Teaching the LLM how to prompt is difficult:** It's possible to instruct the LLM to generate prompts without the LLM knowing what Stable Diffusion is; giving it the exact format to generate a prompt with has decent results. Unfortunately, the often injects plot details or non-visual content into the prompts, no matter how often you tell it not to. These details skew the relevance of the prompts and negatively impact the quality of the generated images.

## **What Did Work: Function Calling Capabilities**
## **What is OpenAI Function Calling?**
On June 13th, OpenAI announced a huge update to the chat completions API - function calling!. This means we can provide the chat model with a function, and the chat model will output a JSON object according to that function's parameters.

Now, the chat models can interpret natural language input into a structured format suitable for external tools, APIs, or database queries. The chat models are designed to detect when a function needs to be called based on the user's input and can then respond with JSON that conforms to the described function's signature.

In essence, **function calling** *is a way to bridge the gap between unstructured language input and structured, actionable output that other systems, tools, or services can use.*

---

## **How FableForge Uses Functions**
For our Stable Diffusion prompts, we need structured data that strictly adheres to specific rules - a function is perfect for that! Let's take a look at one of the functions we used:

In [ ]:
get_visual_description_function = [{
    'name': 'get_passage_setting',
    'description': 'Generate and describe the visuals of a passage in a book. Visuals only, no characters, plot, or people.',
    'parameters': {
        'type': 'object',
        'properties': {
            'setting': {
                'type': 'string',
                'description': 'The visual setting of the passage, e.g. a green forest in the pacific northwest',
            },
            'time_of_day': {
                'type': 'string',
                'description': 'The time of day of the passage, e.g. nighttime, daytime. If unknown, leave blank.',
            },
            'weather': {
                'type': 'string',
                'description': 'The weather of the passage, eg. rain. If unknown, leave blank.',
            },
            'key_elements': {
                'type': 'string',
                'description': 'The key visual elements of the passage, eg tall trees',
            },
            'specific_details': {
                'type': 'string',
                'description': 'The specific visual details of the passage, eg moonlight',
            }
        },
        'required': ['setting', 'time_of_day', 'weather', 'key_elements', 'specific_details']
    }
}]


With this, we can send the chat model a page from our book, the function, and instructions to infer the details from the provided page. In return, we get structured data that we can use to form a great Stable Diffusion prompt!

---

## **LangChain and OpenAI Function Calling**
When we created FableForge, OpenAI announced the new function calling capabilities. Since then, **LangChain** - the open-source library we use to interact with OpenAI's Large Language Models - has added even better support for using functions. Our implementation of functions using LangChain is as follows:

- **Define our function:** First, we define our function, as we did above with `get_visual_description_function`.

- **Give the chat model access to our function:** Next, we call our chat model, including our function within the functions parameter, like so:

In [ ]:

response= self.chat([HumanMessage(content=f'{page}')],functions=get_visual_description_function)

- **Parse the JSON object:** When the chat model uses our function, it provides the output as a JSON object. To convert the JSON object into a Python dictionary containing the function output, we can do the following:

In [ ]:
function_dict = json.loads(response.additional_kwargs['function_call']['arguments'])

In the function, we defined earler, 'setting' was one of the parameters. To access this, we can write:

In [ ]:
setting = function_dict['setting']

And we're done! We can follow the same steps for the each of the other parameters to extract them.

---

## **Perfecting the Process: Using Deep Lake for Storage and Analysis**
The final step breakthrough for perfecting FableForge was using Deep Lake to store the generated images and text. With Deep Lake, we could store multiple modalities of data, such as images and text, in the cloud. The web-based UI provided by Deep Lake made it incredibly straightforward to display, analyze, and optimize the generated images and prompts, improving the quality of our picture book output. For future Stable Diffusion endeavors, we now have a decently-sized dataset showing us what prompts work, and what prompts don't!

---

## **Building FableForge**
FableForge's open-sourced code is located [here](https://github.com/e-johnstonn/FableForge).

FableForge consists of four main components:

**1.** The generation of the text and images

**2.** The combining of the text and images to create the book

**3.** Saving the images and prompts to the Deep Lake dataset

**4.** The UI

Let's take a look at each component individually, starting with the generation of the text and images. Here's a high-level overview of the architecture:

<div align="center">
  <img src="Screenshot 2023-07-13 121544.png" alt="Deep Lake Diagram" width="800" />
</div>